This project dives into the intriguing connection between weather conditions and electric vehicle (EV) charging behavior. It utilizes data from two key sources: the usage records of EV charging stations across the United Kingdom, retrieved from the National Charge Point Registry (NCRP), and historical weather data sourced from the World Weather Online API.

The NCRP dataset, spanning from January 2016 to December 2019, provides detailed insight into each charging session's specifics, including location, date, time, and energy consumed. The World Weather Online API complements this by offering weather information corresponding to the charging sessions' exact time and location.

Through Python's data manipulation capabilities, the project seamlessly merges the charging records with the relevant weather data, creating a comprehensive GeoDataFrame. This combined dataset invites a deeper exploration of the relationship between weather and EV charging patterns, laying a solid foundation for insightful conclusions.

In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely import wkt
import requests
import time
import json
from datetime import datetime, timedelta

This method is a constructor for a class in Python, although the class itself is not 
specified in the provided code. A constructor is a special kind of method in a class that 
is automatically called when an object of that class is created. 

Here's a breakdown of what this specific constructor does: 

• init_(self, api_key): This is the definition of the constructor. It takes two 
parameters: self and api_key. self is a reference to the instance of the class and is 
automatically passed, api_key is a parameter that needs to be provided when 
creating an instance of this class. 

• self. api_key = api_key: This line of code sets the api_key attribute of the 
instance to the api_key parameter provided when the instance was created. This 
api_key is likely used to authenticate with a web API. 
self. base url 
"https://api.weatherapi.com/vl/history.json": This line 

• sets the base_url attribute of the instance to a specific URL This URL is the endpoint 
of a web API that provides historical weather data in JSON format 

In summary, this constructor is used to create an instance of a class that is set up to 
interact with a specific web API. The api_key needed to authenticate with the API and 
the base_url of the API endpoint are both set as attributes of the class instance. 

This method, get_weather, is designed to retrieve weather data for a specific 
geographic location (latitude and longitude) on a specified date. Here's a step-by- 
step breakdown: 

• It accepts three parameters: date, lat, Ion. date is the date for which the 
weather data is to be retrieved. lat and Ion represent the latitude and longitude 
of the location, respectively. 

• It creates a dictionary called params which holds the parameters to be passed in 
the HTTP request. The parameters include: 

• key: the API key used to authenticate with the weather data service. 

• q: the query parameter, which is a string formatted as "latitude,longitude". 

• dt: the date for which the weather data is to be retrieved, formatted as "YYYY- 
MM-DD". 

• It sends a GET request to the base URL of the weather service API (stored in 
self. base url), passing the params dictionary as the parameters of the request. 

• The response from the API is received and converted from a JSON format into a 
Python dictionary using the json( ) method. 

• Finally, the method retums this dictionary which contains the weather data. 

In [2]:
# class WeatherAPI:
#     def __init__(self, api_key):
#         self.api_key = api_key
#         self.base_url = "https://api.weatherapi.com/v1/history.json"

#     def get_weather(self, date, lat, lon):
#         params = {
#             "key": self.api_key,
#             "q": f"{lat},{lon}",
#             "dt": date.strftime("%Y-%m-%d")
#         }
#         response = requests.get(self.base_url, params=params)
#         data = response.json()
#         return data

# def load_gdfs():
#     cwd = os.getcwd()
#     df_path = os.path.join(cwd, '..', 'data', 'interim', 'train_gdf_forward_geocoded.csv')
#     df = pd.read_csv(df_path)
#     df['geometry'] = df['geometry'].apply(wkt.loads)
#     gdf = gpd.GeoDataFrame(df, geometry='geometry')
#     return gdf

# def add_weather_to_gdf(gdf, weather_api):
#     gdf['End DateTime'] = pd.to_datetime(gdf['End DateTime'])
#     weather_data = []
#     for i, row in gdf.iterrows():
#         date = row['End DateTime'].date()
#         lat = row['geometry'].y
#         lon = row['geometry'].x
#         weather = weather_api.get_weather(date, lat, lon)
#         weather_data.append(weather)
#         if i % 500 == 0:
#             print(f"Completed {i} rows")
#             time.sleep(1)
#         if i % 1000 == 0 and i > 0:
#             cwd = os.getcwd()
#             gdf.to_csv(os.path.join(cwd, '..', 'data', 'interim', 'weather', f'gdf_save_{i}.csv'))
#             print(f"Saved at row {i}")
#     gdf['weather'] = weather_data
#     return gdf

# def load_last_saved():
#     cwd = os.getcwd()
#     saves_dir = os.path.join(cwd, '..', 'data', 'interim', 'weather')
#     files = os.listdir(saves_dir)
#     saves = [file for file in files if file.startswith('gdf_save_')]
#     if not saves:
#         return None
#     last_save = max(saves, key=lambda x: os.path.getctime(os.path.join(saves_dir, x)))
#     df = pd.read_csv(os.path.join(saves_dir, last_save))
#     df['geometry'] = df['geometry'].apply(wkt.loads)
#     gdf = gpd.GeoDataFrame(df, geometry='geometry')
#     return gdf

# gdf = load_last_saved()
# if gdf is None:
#     gdf = load_gdfs()
# weather_api = WeatherAPI('3c59adbd6de24fbd9ee153056231007')
# gdf = add_weather_to_gdf(gdf, weather_api)

In [3]:
class WeatherAPI:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://api.weatherapi.com/v1/history.json"

    def get_weather(self, date, country="Scotland"):
        params = {
            "key": self.api_key,
            "q": country,
            "dt": date.strftime("%Y-%m-%d")
        }
        response = requests.get(self.base_url, params=params)
        data = response.json()
        return data

def load_gdfs():
    cwd = os.getcwd()
    df_path = os.path.join(cwd, '..', 'data', 'interim', 'train_gdf_forward_geocoded.csv')
    df = pd.read_csv(df_path)
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    return gdf

def add_weather_to_gdf(gdf, weather_api):
    gdf['End DateTime'] = pd.to_datetime(gdf['End DateTime'])
    weather_data = []
    for i, row in gdf.iterrows():
        date = row['End DateTime'].date()
        weather = weather_api.get_weather(date)
        print(weather)  # Debug line to print the fetched weather data
        weather_data.append(weather)
        if i % 500 == 0:
            print(f"Completed {i} rows")
            time.sleep(1)
        if i % 1000 == 0 and i > 0:
            cwd = os.getcwd()
            gdf.to_csv(os.path.join(cwd, '..', 'data', 'interim', 'weather', f'gdf_save_{i}.csv'))
            print(f"Saved at row {i}")
    gdf['temperature'] = [data['forecast']['forecastday'][0]['day']['avgtemp_c'] if 'forecast' in data else None for data in weather_data]
    gdf['humidity'] = [data['forecast']['forecastday'][0]['day']['avghumidity'] if 'forecast' in data else None for data in weather_data]
    gdf['precipitation'] = [data['forecast']['forecastday'][0]['day']['totalprecip_mm'] if 'forecast' in data else None for data in weather_data]
    return gdf

def load_last_saved():
    cwd = os.getcwd()
    saves_dir = os.path.join(cwd, '..', 'data', 'interim', 'weather')
    files = os.listdir(saves_dir)
    saves = [file for file in files if file.startswith('gdf_save_')]
    if not saves:
        return None
    last_save = max(saves, key=lambda x: os.path.getctime(os.path.join(saves_dir, x)))
    df = pd.read_csv(os.path.join(saves_dir, last_save))
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    return gdf

gdf = load_last_saved()
if gdf is None:
    gdf = load_gdfs()
# weather_api = WeatherAPI('3c59adbd6de24fbd9ee153056231007')
weather_api = WeatherAPI('a26daa0ec9d6403c992152453231007')
gdf = add_weather_to_gdf(gdf, weather_api)

{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
Completed 0 rows
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key has been disabled.'}}
{'error': {'code': 2008, 'message': 'API key 

KeyboardInterrupt: 

In [ ]:
def load_gdfs():
    # Get the current working directory
    cwd = os.getcwd()

    # Define the relative paths to the data files
    weather_path = os.path.join(cwd, '..', 'data', 'interim', 'weather', 'gdf_save_66000.csv')

    # Load the dataframes
    weather = pd.read_csv(weather_path)

    # Convert 'geometry' column to geometry type
    weather['geometry'] = weather['geometry'].apply(wkt.loads)

    # Convert the pandas DataFrames to GeoDataFrames
    weather = gpd.GeoDataFrame(weather, geometry='geometry')
    
    return weather

weather_gdf = load_gdfs()

In [ ]:
weather_gdf.head()

In [ ]:
weather_gdf.shape

In [ ]:
gdf.head()